# Interactive Pipeline Configuration with DAGConfigFactory

This notebook demonstrates the new interactive approach to pipeline configuration using the DAGConfigFactory.
Instead of manually creating 500+ lines of static configuration, we use a guided step-by-step process.

## Workflow Overview

1. **Define Pipeline DAG** - Create the pipeline structure
2. **Initialize DAGConfigFactory** - Set up the interactive factory
3. **Configure Base Settings** - Set shared pipeline configuration
4. **Configure Processing Settings** - Set shared processing configuration
5. **Configure Individual Steps** - Set step-specific configurations
6. **Generate Final Configurations** - Create config instances
7. **Save to JSON** - Export unified configuration file



## Environment Setup

In [1]:
import os
import json
import sys
from pathlib import Path
from datetime import datetime, date
import logging
from typing import List, Optional, Dict, Any


# Set up logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# Get parent directory of current notebook
project_root = str(Path().absolute().parent)
print(f"project root {project_root}")
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"add project root {project_root} into system")

project root /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines
add project root /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines into system


In [2]:
# SageMaker and SAIS imports
from sagemaker import Session
from sagemaker.workflow.pipeline_context import PipelineSession

2025-11-10 04:09:41,422 - INFO - Note: NumExpr detected 48 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2025-11-10 04:09:41,422 - INFO - NumExpr defaulting to 8 threads.
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2025-11-10 04:09:42,560 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
print(f"Role: {PipelineSession().get_caller_identity_arn()}")

2025-11-10 04:09:42,907 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Role: arn:aws:iam::178936618742:role/AmazonSageMaker-ExecutionRole-Default


In [4]:
bucket = "buyer-seller-messaging-reversal"
print(f"Bucket: {bucket}")

Bucket: buyer-seller-messaging-reversal


## Step 1: Define Pipeline DAG

First, we define the pipeline structure using a DAG (Directed Acyclic Graph).
This replaces the hardcoded pipeline structure from the legacy approach.

In [5]:
from cursus.api.dag.base_dag import PipelineDAG


def create_bedrock_batch_pytorch_with_label_ruleset_e2e_dag() -> PipelineDAG:
    """
    Create a DAG for Bedrock Batch-enhanced PyTorch E2E pipeline with Label Ruleset steps.

    This DAG represents a complete end-to-end workflow that uses:
    1. Bedrock prompt template generation and batch processing for LLM-enhanced data
    2. Label ruleset generation and execution for transparent label transformation
    3. PyTorch training, followed by calibration, packaging, and registration

    The label ruleset steps sit between Bedrock processing and training/evaluation,
    providing transparent, rule-based label transformation that's easy to modify.

    Returns:
        PipelineDAG: The directed acyclic graph for the pipeline
    """
    dag = PipelineDAG()

    # Add all nodes - incorporating Bedrock batch processing and label ruleset steps
    dag.add_node("DummyDataLoading_training")  # Dummy data load for training
    dag.add_node("TabularPreprocessing_training")  # Tabular preprocessing for training
    dag.add_node(
        "BedrockPromptTemplateGeneration"
    )  # Bedrock prompt template generation (shared)
    dag.add_node(
        "BedrockBatchProcessing_training"
    )  # Bedrock batch processing step for training
    dag.add_node(
        "LabelRulesetGeneration"
    )  # Label ruleset generation (shared for training and calibration)
    dag.add_node(
        "LabelRulesetExecution_training"
    )  # Label ruleset execution for training data
    # dag.add_node("PyTorchTraining")  # PyTorch training step
    # dag.add_node(
    #    "ModelCalibration_calibration"
    # )  # Model calibration step with calibration variant
    # dag.add_node("Package")  # Package step
    # dag.add_node("Registration")  # MIMS registration step
    # dag.add_node("Payload")  # Payload step
    # dag.add_node("DummyDataLoading_calibration")  # Dummy data load for calibration
    # dag.add_node(
    #    "TabularPreprocessing_calibration"
    # )  # Tabular preprocessing for calibration
    # dag.add_node(
    #    "BedrockBatchProcessing_calibration"
    # )  # Bedrock batch processing step for calibration
    # dag.add_node(
    #    "LabelRulesetExecution_calibration"
    # )  # Label ruleset execution for calibration data
    # dag.add_node("PyTorchModelEval_calibration")  # Model evaluation step

    # Training flow with Bedrock batch processing and label ruleset integration
    dag.add_edge("DummyDataLoading_training", "TabularPreprocessing_training")

    # Bedrock batch processing flow for training - two inputs to BedrockBatchProcessing_training
    dag.add_edge(
        "TabularPreprocessing_training", "BedrockBatchProcessing_training"
    )  # Data input
    dag.add_edge(
        "BedrockPromptTemplateGeneration", "BedrockBatchProcessing_training"
    )  # Template input

    # Label ruleset execution for training - two inputs to LabelRulesetExecution_training
    dag.add_edge(
        "BedrockBatchProcessing_training", "LabelRulesetExecution_training"
    )  # Data input
    dag.add_edge(
        "LabelRulesetGeneration", "LabelRulesetExecution_training"
    )  # Ruleset input

    # Labeled data flows to PyTorch training
    # dag.add_edge("LabelRulesetExecution_training", "PyTorchTraining")

    # Calibration flow with Bedrock batch processing and label ruleset integration
    # dag.add_edge("DummyDataLoading_calibration", "TabularPreprocessing_calibration")

    # Bedrock batch processing flow for calibration - two inputs to BedrockBatchProcessing_calibration
    # dag.add_edge(
    #    "TabularPreprocessing_calibration", "BedrockBatchProcessing_calibration"
    # )  # Data input
    # dag.add_edge(
    #    "BedrockPromptTemplateGeneration", "BedrockBatchProcessing_calibration"
    # )  # Template input

    # Label ruleset execution for calibration - two inputs to LabelRulesetExecution_calibration
    # dag.add_edge(
    #    "BedrockBatchProcessing_calibration", "LabelRulesetExecution_calibration"
    # )  # Data input
    # dag.add_edge(
    #    "LabelRulesetGeneration", "LabelRulesetExecution_calibration"
    # )  # Ruleset input

    # Evaluation flow
    # dag.add_edge("PyTorchTraining", "PyTorchModelEval_calibration")
    # dag.add_edge(
    #    "LabelRulesetExecution_calibration", "PyTorchModelEval_calibration"
    # )  # Use labeled calibration data

    # Model calibration flow - depends on model evaluation
    # dag.add_edge("PyTorchModelEval_calibration", "ModelCalibration_calibration")

    # Output flow
    # dag.add_edge("ModelCalibration_calibration", "Package")
    # dag.add_edge("PyTorchTraining", "Package")  # Raw model is also input to packaging
    # dag.add_edge("PyTorchTraining", "Payload")  # Payload test uses the raw model
    # dag.add_edge("Package", "Registration")
    # dag.add_edge("Payload", "Registration")

    logger.info(
        f"Created Bedrock Batch-PyTorch with Label Ruleset E2E DAG with {len(dag.nodes)} nodes and {len(dag.edges)} edges"
    )
    return dag


# Create the pipeline DAG
dag = create_bedrock_batch_pytorch_with_label_ruleset_e2e_dag()

print(f"Pipeline DAG created with {len(dag.nodes)} steps:")
for node in dag.nodes:
    print(f"  - {node}")

2025-11-10 04:09:43,422 - WARNING - Could not import constants from mods_workflow_core, using local definitions
2025-11-10 04:09:43,781 - INFO - Added node: DummyDataLoading_training
2025-11-10 04:09:43,782 - INFO - Added node: TabularPreprocessing_training
2025-11-10 04:09:43,782 - INFO - Added node: BedrockPromptTemplateGeneration
2025-11-10 04:09:43,783 - INFO - Added node: BedrockBatchProcessing_training
2025-11-10 04:09:43,783 - INFO - Added node: LabelRulesetGeneration
2025-11-10 04:09:43,783 - INFO - Added node: LabelRulesetExecution_training
2025-11-10 04:09:43,783 - INFO - Added edge: DummyDataLoading_training -> TabularPreprocessing_training
2025-11-10 04:09:43,784 - INFO - Added edge: TabularPreprocessing_training -> BedrockBatchProcessing_training
2025-11-10 04:09:43,784 - INFO - Added edge: BedrockPromptTemplateGeneration -> BedrockBatchProcessing_training
2025-11-10 04:09:43,784 - INFO - Added edge: BedrockBatchProcessing_training -> LabelRulesetExecution_training
2025-11

Pipeline DAG created with 6 steps:
  - DummyDataLoading_training
  - TabularPreprocessing_training
  - BedrockPromptTemplateGeneration
  - BedrockBatchProcessing_training
  - LabelRulesetGeneration
  - LabelRulesetExecution_training


## Step 2: Initialize DAGConfigFactory

Now we initialize the DAGConfigFactory with our DAG. This will automatically:
- Map DAG nodes to configuration classes
- Set up the interactive workflow
- Prepare for step-by-step configuration

In [6]:
from cursus.api.factory.dag_config_factory import DAGConfigFactory

# Initialize the factory with our DAG
factory = DAGConfigFactory(dag)

# Get the config class mapping
config_map = factory.get_config_class_map()

print("DAG Node to Config Class Mapping:")
print("=" * 50)
for node_name, config_class in config_map.items():
    print(f"  {node_name:<35} -> {config_class.__name__}")

print(f"\nSuccessfully mapped {len(config_map)} steps to configuration classes.")

2025-11-10 04:09:43,791 - INFO - 🔧 BuilderAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/cursus
2025-11-10 04:09:43,791 - INFO - 🔧 BuilderAutoDiscovery.__init__ - workspace_dirs: []
2025-11-10 04:09:43,792 - INFO - ✅ BuilderAutoDiscovery basic initialization complete
2025-11-10 04:09:43,793 - INFO - ✅ Registry info loaded: 36 steps
2025-11-10 04:09:43,793 - INFO - 🎉 BuilderAutoDiscovery initialization completed successfully
2025-11-10 04:09:43,793 - INFO - 🔍 ScriptAutoDiscovery.__init__ starting - package_root: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/cursus
2025-11-10 04:09:43,794 - INFO - 🔍 ScriptAutoDiscovery.__init__ - workspace_dirs: []
2025-11-10 04:09:43,794 - INFO - 🔍 ScriptAutoDiscovery.__init__ - priority_workspace_dir: None
2025-11-10 04:09:43,794 - INFO - ✅ Registry info loaded: 36 steps
2025-11-10 04:09:43,795 - INFO - 🎉 ScriptAutoDiscovery initialization com

DAG Node to Config Class Mapping:
  DummyDataLoading_training           -> DummyDataLoadingConfig
  TabularPreprocessing_training       -> TabularPreprocessingConfig
  BedrockPromptTemplateGeneration     -> BedrockPromptTemplateGenerationConfig
  BedrockBatchProcessing_training     -> BedrockBatchProcessingConfig
  LabelRulesetGeneration              -> LabelRulesetGenerationConfig
  LabelRulesetExecution_training      -> LabelRulesetExecutionConfig

Successfully mapped 6 steps to configuration classes.


## Step 3: Configure Base Pipeline Settings

These settings are shared across ALL pipeline steps. Instead of repeating them
in every step configuration, we set them once here.

In [7]:
# Get base configuration requirements
base_requirements = factory.get_base_config_requirements()

print("Base Pipeline Configuration Requirements:")
print("=" * 50)
for req in base_requirements:
    marker = "*" if req["required"] else " "
    default_info = (
        f" (default: {req.get('default')})"
        if not req["required"] and "default" in req
        else ""
    )
    print(f"{marker} {req['name']:<25} ({req['type']}){default_info}")
    print(f"    {req['description']}")
    print()

Base Pipeline Configuration Requirements:
* author                    (str)
    Author or owner of the pipeline.

* bucket                    (str)
    S3 bucket name for pipeline artifacts and data.

* role                      (str)
    IAM role for pipeline execution.

* region                    (str)
    Custom region code (NA, EU, FE) for internal logic.

* service_name              (str)
    Service name for the pipeline.

* pipeline_version          (str)
    Version string for the SageMaker Pipeline.

  model_class               (str) (default: xgboost)
    Model class (e.g., XGBoost, PyTorch).

  current_date              (str) (default: PydanticUndefined)
    Current date, typically used for versioning or pathing.

  framework_version         (str) (default: 2.1.0)
    Default framework version (e.g., PyTorch).

  py_version                (str) (default: py310)
    Default Python version.

  source_dir                (Optional) (default: None)
    Common source directory fo

In [8]:
# Set up basic configuration values
region_list = ["NA", "EU", "FE"]
region_selection = 0
region = region_list[region_selection]

# Map region to AWS region
region_mapping = {"NA": "us-east-1", "EU": "eu-west-1", "FE": "us-west-2"}
aws_region = region_mapping[region]

service_name = "BuyerAbuseRnR"
pipeline_version = "0.0.1"
author = "lukexie"
model_class = "pytorch"

# Get current directory and set up paths
current_dir = Path.cwd()
package_root = Path(current_dir).resolve()
source_dir = Path("docker")
project_root_folder = "rnr_pytorch_bedrock"

# Set base configuration
factory.set_base_config(
    # Infrastructure settings
    bucket=bucket,
    role=PipelineSession().get_caller_identity_arn(),
    region=region,
    aws_region=aws_region,
    # Project identification
    author=author,
    service_name=service_name,
    pipeline_version=pipeline_version,
    model_class=model_class,
    # Framework settings
    framework_version="2.1.0",
    py_version="py310",
    source_dir=str(source_dir),
    project_root_folder=project_root_folder,
    # Date settings
    current_date=date.today().strftime("%Y-%m-%d"),
    # Enable Cache
    enable_caching=False,
    # Use Secure PyPI
    use_secure_pypi=False,
)

print("✅ Base pipeline configuration set successfully!")
print(f"   Region: {region} ({aws_region})")
print(f"   Service: {service_name}")
print(f"   Author: {author}")
print(f"   Pipeline Version: {pipeline_version}")

2025-11-10 04:09:44,010 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-11-10 04:09:44,340 - INFO - Base configuration set successfully


✅ Base pipeline configuration set successfully!
   Region: NA (us-east-1)
   Service: BuyerAbuseRnR
   Author: lukexie
   Pipeline Version: 0.0.1


## Step 4: Configure Base Processing Settings

These settings are shared across all PROCESSING steps (data loading, preprocessing, etc.)
but not training steps.

In [9]:
# Get base processing configuration requirements
processing_requirements = factory.get_base_processing_config_requirements()

if processing_requirements:
    print("Base Processing Configuration Requirements:")
    print("=" * 50)
    for req in processing_requirements:
        marker = "*" if req["required"] else " "
        default_info = (
            f" (default: {req.get('default')})"
            if not req["required"] and "default" in req
            else ""
        )
        print(f"{marker} {req['name']:<30} ({req['type']}){default_info}")
        print(f"    {req['description']}")
        print()
else:
    print("No base processing configuration required for this pipeline.")

Base Processing Configuration Requirements:
  processing_instance_count      (int) (default: 1)
    Instance count for processing jobs

  processing_volume_size         (int) (default: 500)
    Volume size for processing jobs in GB

  processing_instance_type_large (str) (default: ml.m5.4xlarge)
    Large instance type for processing step.

  processing_instance_type_small (str) (default: ml.m5.2xlarge)
    Small instance type for processing step.

  use_large_processing_instance  (bool) (default: False)
    Set to True to use large instance type, False for small instance type.

  processing_source_dir          (Optional) (default: None)
    Source directory for processing scripts. Falls back to base source_dir if not provided.

  processing_entry_point         (Optional) (default: None)
    Entry point script for processing, must be relative to source directory. Can be overridden by derived classes.

  processing_script_arguments    (Optional) (default: None)
    Optional arguments fo

In [10]:
# Set base processing configuration if needed
if processing_requirements:
    processing_source_dir = source_dir / "scripts"

    factory.set_base_processing_config(
        # Processing infrastructure
        processing_source_dir=str(processing_source_dir),
        processing_instance_type_large="ml.m5.12xlarge",
        processing_instance_type_small="ml.m5.4xlarge",
    )

    print("✅ Base processing configuration set successfully!")
    print(f"   Processing source: {processing_source_dir}")
else:
    print("✅ No base processing configuration needed.")

2025-11-10 04:09:44,354 - INFO - Package location discovery succeeded (bundled): /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker
2025-11-10 04:09:44,354 - INFO - Hybrid resolution completed successfully via Package Location Discovery: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/docker
2025-11-10 04:09:44,355 - INFO - Base processing configuration set successfully


✅ Base processing configuration set successfully!
   Processing source: docker/scripts


## Step 5: Check Configuration Status

Let's see which steps still need configuration.

In [11]:
# Check current status
status = factory.get_configuration_status()
pending_steps = factory.get_pending_steps()

print("Configuration Status:")
print("=" * 30)
print(f"Base config set: {'✅' if status['base_config'] else '❌'}")
print(f"Processing config set: {'✅' if status['base_processing_config'] else '❌'}")
print(f"Total steps: {len(config_map)}")
print(f"Pending steps: {len(pending_steps)}")
print()

if pending_steps:
    print("Steps needing configuration:")
    for step in pending_steps:
        print(f"  - {step}")
else:
    print("✅ All steps configured!")

Configuration Status:
Base config set: ✅
Processing config set: ✅
Total steps: 6
Pending steps: 6

Steps needing configuration:
  - DummyDataLoading_training
  - TabularPreprocessing_training
  - BedrockPromptTemplateGeneration
  - BedrockBatchProcessing_training
  - LabelRulesetGeneration
  - LabelRulesetExecution_training


## Step 6: Configure Individual Steps

Now we configure each step with its specific requirements. The factory will show us
only the fields that are unique to each step (not inherited from base configs).

### Step 6.1 Prompt Generation

In [12]:
config_requirements = factory.get_step_requirements("BedrockPromptTemplateGeneration")

if config_requirements:
    print("BedrockPromptTemplateGeneration Config Requirements:")
    print("=" * 50)
    for req in config_requirements:
        marker = "*" if req["required"] else " "
        default_info = (
            f" (default: {req.get('default')})"
            if not req["required"] and "default" in req
            else ""
        )
        print(f"{marker} {req['name']:<30} ({req['type']}){default_info}")
        print(f"    {req['description']}")
        print()
else:
    print("No base processing configuration required for this pipeline.")

BedrockPromptTemplateGeneration Config Requirements:
* input_placeholders             (List)
    List of input field names to include in the template (e.g., ['input_data', 'context', 'metadata'])

  prompt_configs_path            (str) (default: prompt_configs)
    Subdirectory name or relative path under the processing source directory for prompt configuration files (system_prompt.json, output_format.json, instruction.json, category_definitions.json). Must be a relative path, not absolute. Examples: 'prompt_configs', 'docker/prompt_configs', 'config/prompts'

  template_task_type             (str) (default: classification)
    Type of task for template generation (classification, sentiment_analysis, content_moderation)

  template_style                 (str) (default: structured)
    Style of template generation (structured, conversational, technical)

  validation_level               (str) (default: standard)
    Level of template validation (basic, standard, comprehensive)

  output

In [13]:
from cursus.steps.configs.config_bedrock_prompt_template_generation_step import (
    BedrockPromptTemplateGenerationConfig,
    SystemPromptConfig,
    OutputFormatConfig,
    InstructionConfig,
)

In [14]:
category_definitions_path = "prompt_configs"

In [15]:
system_prompt_settings = SystemPromptConfig(
    role_definition="expert in analyzing buyer-seller messaging conversations and shipping logistics",
    expertise_areas=[
        "buyer-seller messaging analysis",
        "shipping logistics",
        "delivery timing analysis",
        "e-commerce dispute resolution",
        "classification and categorization",
    ],
    responsibilities=[
        "classify interactions based on message content",
        "analyze shipping events and delivery timing",
        "categorize into predefined dispute categories",
        "provide evidence-based reasoning for classifications",
    ],
    behavioral_guidelines=[
        "be precise in classification decisions",
        "be objective in evidence evaluation",
        "be thorough in timeline analysis",
        "follow exact formatting requirements",
        "consider all available evidence sources",
    ],
    tone="professional",  # Options: "professional", "casual", "technical", "formal"
)

In [16]:
output_format_settings = OutputFormatConfig(
    format_type="structured_json",
    required_fields=["category", "confidence_score", "key_evidence", "reasoning"],
    # Human-readable descriptions for prompt generation
    field_descriptions={
        "category": "Exactly one category from the predefined list (case-sensitive match required)",
        "confidence_score": "Decimal number between 0.00 and 1.00 indicating classification certainty",
        "key_evidence": "Object containing three arrays: message_evidence, shipping_evidence, timeline_evidence",
        "reasoning": "Object containing three arrays: primary_factors, supporting_evidence, contradicting_evidence",
    },
    # Machine-readable JSON schema for validation schema generation
    json_schema={
        "type": "object",
        "properties": {
            "category": {
                "type": "string",
                "enum": [],  # Will be populated from category_definitions.json
                "description": "Exactly one category from the predefined list (case-sensitive match required)",
            },
            "confidence_score": {
                "type": "number",
                "minimum": 0.0,
                "maximum": 1.0,
                "description": "Decimal number between 0.00 and 1.00 indicating classification certainty",
            },
            "key_evidence": {
                "type": "object",
                "description": "Object containing three arrays of evidence from different sources",
                "properties": {
                    "message_evidence": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Direct quotes from dialogue with speaker identification",
                    },
                    "shipping_evidence": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Tracking events with timestamps",
                    },
                    "timeline_evidence": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Chronological sequence of key events",
                    },
                },
                "required": [
                    "message_evidence",
                    "shipping_evidence",
                    "timeline_evidence",
                ],
            },
            "reasoning": {
                "type": "object",
                "description": "Object containing three arrays explaining the classification decision",
                "properties": {
                    "primary_factors": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Main reasons supporting the selected category",
                    },
                    "supporting_evidence": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Additional evidence that strengthens the classification",
                    },
                    "contradicting_evidence": {
                        "type": "array",
                        "items": {"type": "string"},
                        "description": "Evidence that contradicts the classification (use empty array if none)",
                    },
                },
                "required": [
                    "primary_factors",
                    "supporting_evidence",
                    "contradicting_evidence",
                ],
            },
        },
        "required": ["category", "confidence_score", "key_evidence", "reasoning"],
        "additionalProperties": False,
    },
    formatting_rules=[
        # Core JSON requirements
        "Output MUST be valid, parseable JSON.",
        "Do not include any text before the opening { or after the closing }.",
        "CRITICAL: Do NOT wrap JSON in markdown code blocks - no ``` or ```json markers.",
        "CRITICAL: Output pure JSON starting with { and ending with } - nothing else.",
        "Ensure all arrays and objects are properly closed.",
        "Use empty arrays [] for missing values, not null or empty strings.",
        "Do not include trailing commas.",
        "Ensure proper escaping of special characters in strings.",
        # Quote handling – JSON structure vs content
        "Quote Handling - JSON Structure: ALWAYS use ASCII double quotes (U+0022) for JSON keys "
        "and for starting/ending JSON string values.",
        "Quote Handling - Cited Content (Smart Quotes): If the input text contains any non-ASCII or "
        "'smart' quotation marks, do NOT copy those characters into the JSON. Instead, replace each of "
        "them with a simple ASCII apostrophe (').",
        "The smart quotation marks that MUST be replaced include (but are not limited to): "
        "U+201C (LEFT DOUBLE QUOTATION MARK), U+201D (RIGHT DOUBLE QUOTATION MARK), "
        "U+201E (DOUBLE LOW-9 QUOTATION MARK, used in German), U+201F, and "
        "U+2018/U+2019/U+201A/U+201B (Unicode single quotation marks).",
        "Example (described, not shown verbatim): If the input shows a name surrounded by any kind of "
        "non-ASCII quote characters (for example, a German-style opening quote before a person name), "
        "you MUST output that name in JSON as 'Person Name' using ASCII apostrophes only.",
        # NEW: avoid unescaped inner double quotes (the 8\"x10\" / \"impossible\" problem)
        "Inner Double Quotes - General Rule: Inside JSON string values, NEVER include a raw ASCII "
        'double quote character (") as part of the content. The ONLY double quotes you should ever '
        "emit are the ones required by JSON syntax (around keys and around entire string values).",
        "Inner Double Quotes - Rewriting Measurements: If the original text uses double quotes for "
        'measurements or sizes (for example, a rug described as 8"x10"), you MUST rewrite this in '
        "plain text without inner double quotes (for example, '8 x 10 inch rug' or '8 by 10 inch rug').",
        "Inner Double Quotes - Rewriting Emphasized Phrases: If the original text puts a word or "
        "phrase in double quotes (for example, a buyer writing a word like impossible in quotes), "
        "you MUST rewrite it using apostrophes or plain text, such as [BUYER]: 'impossible' or "
        "[BUYER]: impossible, without any inner double quote characters.",
        "Inner Double Quotes - No Literal Copy: Do NOT copy any internal double quotes from the source "
        "text into the JSON output. When the original text uses double quotes around a phrase, output "
        "that phrase using ASCII apostrophes (') or rephrase it so that no internal double quotes appear "
        "inside the JSON string.",
        # Summary rule
        "Summary: Use the ASCII double quote character (U+0022) only for JSON structure "
        "(keys and string delimiters). Inside JSON string values, never emit smart quotation marks "
        "and never emit raw inner double quotes. When the original text uses quotes around a phrase "
        "or measurement, rewrite that phrase using ASCII apostrophes (') or plain words instead.",
    ],
    validation_requirements=[
        "Must be valid JSON format",
        "Category must match exactly from predefined list",
        "Confidence score must be number between 0.0 and 1.0",
        "All required fields must be present",
        "key_evidence and reasoning must be objects with nested arrays",
    ],
    evidence_validation_rules=[
        "Message Evidence must include direct quotes with speaker identification",
        "Shipping Evidence must include tracking events with timestamps",
        "Timeline Evidence must show chronological sequence of events",
        "All evidence must reference specific content from input data",
    ],
    example_output={
        "category": "TrueDNR",
        "confidence_score": 0.92,
        "key_evidence": {
            "message_evidence": [
                "[BUYER]: Hello, I have not received my package, but I see the order shows that it has been delivered, why?",
                "[BUYER]: But I did not find any package, please refund me, thank you",
            ],
            "shipping_evidence": [
                "[Event Time]: 2025-02-21T17:40:49.323Z [Ship Track Event]: Delivered to customer",
                "No further shipping events after delivery confirmation",
            ],
            "timeline_evidence": [
                "Delivery confirmation on 2025-02-21 17:40",
                "Buyer reports non-receipt starting 2025-02-25 07:14",
            ],
        },
        "reasoning": {
            "primary_factors": [
                "Tracking shows package was delivered successfully",
                "Buyer explicitly states they did not receive the package after delivery scan",
            ],
            "supporting_evidence": [
                "Buyer requests refund due to missing package",
                "No evidence of buyer receiving wrong/defective item",
            ],
            "contradicting_evidence": [],
        },
    },
)

In [17]:
instruction_settings = InstructionConfig(
    include_analysis_steps=True,
    include_decision_criteria=True,
    include_reasoning_requirements=True,
    step_by_step_format=True,
    include_evidence_validation=True,
    classification_guidelines={
        "sections": [
            {
                "title": "## Classification Guidelines",
                "subsections": [
                    {
                        "title": "### 1. Output Format Requirements",
                        "content": [
                            "**Category Selection:**",
                            "- Choose exactly ONE category from the provided list",
                            "- Category name must match exactly (case-sensitive)",
                            "",
                            "**Confidence Score:**",
                            "- Provide as decimal number between 0.00 and 1.00 (e.g., 0.95)",
                            "- Base confidence for complete data: 0.7-1.0",
                            "- Missing one field: reduce by 0.1-0.2",
                            "- Missing two fields: reduce by 0.2-0.3",
                            "- Minimum confidence threshold: 0.5",
                        ],
                    },
                    {
                        "title": "### 2. Shiptrack Parsing Rules",
                        "content": [
                            "**Multiple Shipment Structure:**",
                            "- Multiple shipment sequences separated by shipment IDs",
                            '- Each sequence starts with "[bom] [Shipment ID]:* [eom]"',
                            "",
                            "**Analysis Approach:**",
                            "- Process each shipment sequence separately",
                            "- Compare delivery events (EVENT_301) across all sequences",
                            "",
                            "**Key Event Codes:**",
                            "- EVENT_301: Delivery confirmation",
                            "- EVENT_302: Out for delivery",
                            "- EVENT_201: Arrival at facility",
                        ],
                    },
                    {
                        "title": "### 3. Missing Data Handling",
                        "content": [
                            "**When Dialogue is Empty but Shiptrack Exists:**",
                            "- Focus on shipping events and timeline",
                            "- Reduce confidence score by 0.1-0.2",
                            "",
                            "**When Shiptrack is Empty but Dialogue Exists:**",
                            "- Focus on message content and reported issues",
                            "- Reduce confidence score by 0.1-0.2",
                        ],
                    },
                    {
                        "title": "### 4. Category Priority Hierarchy",
                        "content": [
                            "**Tier 1: Abuse Pattern Categories (Highest Priority)**",
                            "- PDA_Undeliverable: Verify no delivery + refund given",
                            "- PDA_Early_Refund: Verify refund before delivery",
                            "",
                            "**Tier 2: Delivery Status Categories**",
                            "- TrueDNR: Delivered but disputed",
                            "- Confirmed_Delay: External factors confirmed",
                        ],
                    },
                    {
                        "title": "### 5. Evidence Requirements",
                        "content": [
                            "**Message Evidence Must Include:**",
                            "- Direct quotes from dialogue with speaker identification",
                            "",
                            "**Shipping Evidence Must Include:**",
                            "- All tracking events listed chronologically",
                            "",
                            "**Timeline Evidence Must Show:**",
                            "- Clear chronological sequence of events",
                        ],
                    },
                ],
            }
        ]
    },
)

In [18]:
category_definitions_path

'prompt_configs'

In [19]:
step_name = "BedrockPromptTemplateGeneration"
factory.set_step_config(
    step_name,
    # input
    input_placeholders=[
        "dialogue",
        "shiptrack_event_history_by_order",
        "shiptrack_max_estimated_arrival_date_by_order",
    ],
    prompt_configs_path=str(category_definitions_path),
    # basic setting
    template_task_type="buyer_seller_classification",
    template_style="structured",
    validation_level="comprehensive",
    template_version="2.0",
    # Output configuration
    output_format_type="structured_json",
    required_output_fields=[
        "Category",
        "Confidence Score",
        "Key Evidence",
        "Reasoning",
    ],
    # Template features
    include_examples=True,
    generate_validation_schema=True,
    # Sub-configurations (Pydantic models)
    system_prompt_settings=system_prompt_settings,
    output_format_settings=output_format_settings,
    instruction_settings=instruction_settings,
    processing_entry_point="bedrock_prompt_template_generation.py",
)
print(f"✅ {step_name} configured")

2025-11-10 04:09:44,423 - INFO - Generated system prompt config: docker/scripts/prompt_configs/system_prompt.json
2025-11-10 04:09:44,424 - INFO - Generated output format config: docker/scripts/prompt_configs/output_format.json
2025-11-10 04:09:44,425 - INFO - Generated instruction config: docker/scripts/prompt_configs/instruction.json
2025-11-10 04:09:44,425 - INFO - Skipping category_definitions.json generation (no category definitions available)
2025-11-10 04:09:44,425 - INFO - Generated prompt configuration bundle in: docker/scripts/prompt_configs
2025-11-10 04:09:44,426 - INFO - Bundle contains 3 JSON configuration files: system_prompt.json, output_format.json, instruction.json
2025-11-10 04:09:44,426 - INFO - Auto-generated prompt configuration bundle at: docker/scripts/prompt_configs
2025-11-10 04:09:44,427 - INFO - ✅ BedrockPromptTemplateGeneration configured successfully using BedrockPromptTemplateGenerationConfig


✅ BedrockPromptTemplateGeneration configured


### Step 6.2: Configure Dummy Data Loading Steps

In [20]:
# Configure dummy data loading
if "DummyDataLoading_training" in pending_steps:
    step_name = "DummyDataLoading_training"

    data_source = (
        "s3://buyer-seller-messaging-reversal/production-pipeline/raw-input/2025-11-03"
    )

    factory.set_step_config(
        step_name,
        job_type="training",
        data_source=data_source,
        processing_entry_point="dummy_data_loading.py",
        use_large_processing_instance=True,
        write_data_shards=True,
        output_format="PARQUET",
    )
    print(f"✅ {step_name} configured")

2025-11-10 04:09:44,431 - INFO - ✅ DummyDataLoading_training configured successfully using DummyDataLoadingConfig


✅ DummyDataLoading_training configured


### Step 6.3: Configure Preprocessing Steps

In [21]:
# Configure training preprocessing
if "TabularPreprocessing_training" in pending_steps:
    step_name = "TabularPreprocessing_training"

    factory.set_step_config(
        step_name,
        job_type="training",
        label_name="reversal_flag",
        processing_entry_point="tabular_preprocessing.py",
        use_large_processing_instance=True,
    )
    print(f"✅ {step_name} configured")

2025-11-10 04:09:44,439 - INFO - ✅ TabularPreprocessing_training configured successfully using TabularPreprocessingConfig


✅ TabularPreprocessing_training configured


### Step 6.4: Configure Remaining Steps

**USER INPUT BLOCK**: Fill in the essential fields for each remaining step.
The factory has identified the required fields for each step.

In [22]:
# Get current pending steps
current_pending = factory.get_pending_steps()

print("Remaining steps to configure:")
print("=" * 40)

for step_name in current_pending:
    requirements = factory.get_step_requirements(step_name)
    essential_reqs = [req for req in requirements if req["required"]]

    print(f"\n{step_name}:")
    print(f"  Essential fields ({len(essential_reqs)}):")
    for req in essential_reqs:
        print(f"    * {req['name']} ({req['type']}) - {req['description']}")

    if len(requirements) > len(essential_reqs):
        optional_count = len(requirements) - len(essential_reqs)
        print(f"  Optional fields: {optional_count}")

Remaining steps to configure:

BedrockBatchProcessing_training:
  Essential fields (1):
    * bedrock_batch_role_arn (str) - IAM role ARN for batch inference jobs (e.g., 'arn:aws:iam::123456789012:role/BedrockBatchRole'). Must have permissions for Bedrock batch inference and S3 access.
  Optional fields: 20

LabelRulesetGeneration:
  Essential fields (2):
    * label_config (LabelConfig) - Label configuration defining output label structure and valid values
    * rule_definitions (RulesetDefinitionList) - List of rule definitions for classification
  Optional fields: 5

LabelRulesetExecution_training:
  Essential fields (1):
    * job_type (str) - One of ['training','validation','testing','calibration'] - determines which splits to process (REQUIRED)
  Optional fields: 4


In [23]:
config_requirements = factory.get_step_requirements("BedrockBatchProcessing_training")

if config_requirements:
    print("BedrockPromptTemplateGeneration Config Requirements:")
    print("=" * 50)
    for req in config_requirements:
        marker = "*" if req["required"] else " "
        default_info = (
            f" (default: {req.get('default')})"
            if not req["required"] and "default" in req
            else ""
        )
        print(f"{marker} {req['name']:<30} ({req['type']}){default_info}")
        print(f"    {req['description']}")
        print()
else:
    print("No base processing configuration required for this pipeline.")

BedrockPromptTemplateGeneration Config Requirements:
* bedrock_batch_role_arn         (str)
    IAM role ARN for batch inference jobs (e.g., 'arn:aws:iam::123456789012:role/BedrockBatchRole'). Must have permissions for Bedrock batch inference and S3 access.

  job_type                       (str) (default: training)
    One of ['training','validation','testing','calibration'] - determines processing behavior and output naming

  bedrock_primary_model_id       (str) (default: anthropic.claude-sonnet-4-5-20250929-v1:0)
    Primary Bedrock model ID for processing (Claude Sonnet 4.5 default, latest stable model)

  bedrock_fallback_model_id      (Optional) (default: anthropic.claude-sonnet-4-20250514-v1:0)
    Fallback model ID for inference profile failures (Claude Sonnet 4.0 for production reliability)

  bedrock_inference_profile_arn  (Optional) (default: None)
    Inference profile ARN for capacity management (e.g., 'arn:aws:bedrock:us-east-1:123456789012:inference-profile/abc123')

  

In [24]:
bedrock_batch_role_arn = PipelineSession().get_caller_identity_arn()

2025-11-10 04:09:44,472 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [25]:
bedrock_primary_model_id = "anthropic.claude-sonnet-4-5-20250929-v1:0"

In [26]:
bedrock_inference_profile_arn = "arn:aws:bedrock:us-east-1:178936618742:inference-profile/us.anthropic.claude-sonnet-4-20250514-v1:0"

In [27]:
bedrock_max_records_per_job = 30000

In [28]:
# Configure Model Evaluation
if "BedrockBatchProcessing_training" in current_pending:
    factory.set_step_config(
        "BedrockBatchProcessing_training",
        job_type="training",
        processing_entry_point="bedrock_batch_processing.py",
        bedrock_batch_role_arn=bedrock_batch_role_arn,
        bedrock_primary_model_id=bedrock_primary_model_id,
        bedrock_inference_profile_arn=bedrock_inference_profile_arn,
        bedrock_max_records_per_job=bedrock_max_records_per_job,
    )
    print(f"✅ BedrockBatchProcessing_training configured")

2025-11-10 04:09:44,671 - INFO - ✅ BedrockBatchProcessing_training configured successfully using BedrockBatchProcessingConfig


✅ BedrockBatchProcessing_training configured


### Step 6.5: Configure Label Ruleset Generation Steps

In [29]:
from cursus.steps.configs.config_label_ruleset_generation_step import (
    LabelRulesetGenerationConfig,
    LabelConfig,
    RuleDefinition,
    RulesetDefinitionList,
    RuleCondition,
    ComparisonOperator,
)

In [30]:
label_config = LabelConfig(
    output_label_name="llm_reversal_flag",
    output_label_type="binary",
    label_values=[0, 1],
    label_mapping={"0": "No_Reversal_Required", "1": "Reversal_Required"},
    default_label=1,  # Conservative: flag for review if no rules match
    evaluation_mode="priority",
)

In [31]:
rule_no_reversal = RuleDefinition(
    name="No_Reversal_Categories",
    priority=1,
    enabled=True,
    conditions=RuleCondition(
        field="llm_category",
        operator=ComparisonOperator.IN,
        value=["TrueDNR", "PDA_Undeliverable", "PDA_Early_Refund", "Returnless_Refund"],
    ),
    output_label=0,
    description="Categories indicating NO reversal: delivered/fraud patterns, logistics delays, goodwill refunds",
)

# Rule 2: Categories indicating reversal required (output_label = 1)
rule_reversal_required = RuleDefinition(
    name="Reversal_Required_Categories",
    priority=2,
    enabled=True,
    conditions=RuleCondition(
        field="llm_category",
        operator=ComparisonOperator.IN,
        value=[
            "Confirmed_Delay",
            "Delivery_Attempt_Failed",
            "Seller_Unable_To_Ship",
            "Buyer_Received_WrongORDefective_Item",
            "BuyerCancellation",
            "Return_NoLongerNeeded",
            "Product_Information_Support",
            "Insufficient_Information",
        ],
    ),
    output_label=1,
    description="Categories indicating reversal required: legitimate refunds, quality issues, cancellations, manual review cases",
)

# Collect all rules (just 2 rules now!)
# Wrap rules in RulesetDefinitionList (required by config)
ruleset_definitions = RulesetDefinitionList(
    rules=[rule_no_reversal, rule_reversal_required]
)

In [32]:
if "LabelRulesetGeneration" in pending_steps:
    step_name = "LabelRulesetGeneration"

    factory.set_step_config(
        step_name,
        # Label configuration (Pydantic model)
        label_config=label_config,
        # Rule definitions (RulesetDefinitionList with Pydantic models)
        rule_definitions=ruleset_definitions,
        # ===== Tier 2: Optional Configuration (with defaults) =====
        # Validation settings (all default to True)
        enable_field_validation=True,
        enable_label_validation=True,
        enable_logic_validation=True,
        # Optimization settings (defaults to True)
        enable_rule_optimization=True,
        # Configuration path (defaults to 'ruleset_configs')
        ruleset_configs_path="ruleset_configs",
        # Processing configuration (defaults to 'label_ruleset_generation.py')
        processing_entry_point="label_ruleset_generation.py",
    )

    print(f"✅ {step_name} configured")

2025-11-10 04:09:44,697 - INFO - Generated label config: docker/scripts/ruleset_configs/label_config.json
2025-11-10 04:09:44,698 - INFO - Generated ruleset: docker/scripts/ruleset_configs/ruleset.json
2025-11-10 04:09:44,698 - INFO - Generated ruleset configuration bundle in: docker/scripts/ruleset_configs
2025-11-10 04:09:44,699 - INFO - Bundle contains 2 JSON configuration files: label_config.json, ruleset.json
2025-11-10 04:09:44,699 - INFO - Auto-generated ruleset configuration bundle at: docker/scripts/ruleset_configs
2025-11-10 04:09:44,699 - INFO - ✅ LabelRulesetGeneration configured successfully using LabelRulesetGenerationConfig


✅ LabelRulesetGeneration configured


### Step 6.6: Configure Label Ruleset Execution Steps

In [33]:
from cursus.steps.configs.config_label_ruleset_execution_step import (
    LabelRulesetExecutionConfig,
)

In [34]:
if "LabelRulesetExecution_training" in pending_steps:
    step_name = "LabelRulesetExecution_training"

    factory.set_step_config(
        step_name,
        # ===== Tier 1: Required Configuration =====
        # Job type determines which splits to process
        job_type="training",  # One of: 'training', 'validation', 'testing', 'calibration'
        # ===== Tier 2: Optional Configuration (with defaults) =====
        # Execution configuration (all default to True)
        fail_on_missing_fields=True,  # Fail if llm_category field missing
        enable_rule_match_tracking=True,  # Track which rules match
        enable_progress_logging=True,  # Log progress during execution
        # Data format configuration (defaults to empty string for auto-detection)
        preferred_input_format="CSV",  # Options: 'CSV', 'TSV', 'Parquet', or '' for auto
        # Processing configuration (defaults to 'label_ruleset_execution.py')
        processing_entry_point="label_ruleset_execution.py",
        # ===== Processing Step Base Configuration =====
        # SageMaker instance configuration
        use_large_processing_instance=True,
    )

    print(f"✅ {step_name} configured for job_type='training'")

2025-11-10 04:09:44,710 - INFO - ✅ LabelRulesetExecution_training configured successfully using LabelRulesetExecutionConfig


✅ LabelRulesetExecution_training configured for job_type='training'


## Step 7: Generate Final Configurations

Now that all steps are configured, we can generate the final configuration instances.
The factory will validate that all essential fields are provided and create the config objects.

In [35]:
# Check final status
final_status = factory.get_configuration_status()
final_pending = factory.get_pending_steps()

print("Final Configuration Status:")
print("=" * 40)
print(f"Base config: {'✅' if final_status['base_config'] else '❌'}")
print(f"Processing config: {'✅' if final_status['base_processing_config'] else '❌'}")
print(f"Pending steps: {len(final_pending)}")

if final_pending:
    print("\nStill pending:")
    for step in final_pending:
        print(f"  - {step}")
    print("\n⚠️  Please configure remaining steps before generating configs.")
else:
    print("\n✅ All steps configured! Ready to generate configurations.")

Final Configuration Status:
Base config: ✅
Processing config: ✅
Pending steps: 0

✅ All steps configured! Ready to generate configurations.


In [36]:
# Generate final configurations
if not final_pending:
    try:
        print("Generating final configurations...")
        configs = factory.generate_all_configs()

        print(f"\n✅ Successfully generated {len(configs)} configuration instances:")
        for i, config in enumerate(configs, 1):
            print(f"  {i:2d}. {config.__class__.__name__}")

        print("\n🎉 Configuration generation complete!")

    except Exception as e:
        print(f"\n❌ Configuration generation failed: {e}")
        print("\nPlease check that all required fields are provided.")
        configs = None
else:
    print("\n⚠️  Cannot generate configs - some steps are still pending configuration.")
    configs = None

2025-11-10 04:09:44,723 - INFO - ✅ Returning 6 pre-validated configuration instances


Generating final configurations...

✅ Successfully generated 6 configuration instances:
   1. BedrockPromptTemplateGenerationConfig
   2. DummyDataLoadingConfig
   3. TabularPreprocessingConfig
   4. BedrockBatchProcessingConfig
   5. LabelRulesetGenerationConfig
   6. LabelRulesetExecutionConfig

🎉 Configuration generation complete!


In [37]:
len(configs)

6

## Step 8: Save to JSON

Finally, we save the generated configurations to a unified JSON file using the existing
`merge_and_save_configs` utility. This creates the same format as the legacy approach
but with much less effort!

In [38]:
if configs:
    # Set up output directory and filename
    MODEL_CLASS = "pytorch"
    service_name = "BuyerAbuseRnR"

    config_dir = Path(current_dir) / "pipeline_config"
    config_dir.mkdir(parents=True, exist_ok=True)

    config_file_name = f"config.json"
    config_path = config_dir / config_file_name

    print(f"Saving configurations to: {config_path}")

    # Use the existing merge_and_save_configs utility
    from cursus.steps.configs.utils import merge_and_save_configs

    try:
        merged_config = merge_and_save_configs(configs, str(config_path))

        print(f"\n✅ Configuration saved successfully!")
        print(f"   File: {config_path}")
        print(f"   Size: {config_path.stat().st_size / 1024:.1f} KB")

        # Also save hyperparameters separately (for compatibility)
        # hyperparam_path = source_dir / 'hyperparams' / f'hyperparameters.json'
        # with open(hyperparam_path, 'w') as f:
        #    json.dump(xgb_hyperparams.model_dump(), f, indent=2, sort_keys=True)

        # print(f"   Hyperparameters: {hyperparam_path}")

        # print(f"\n🎉 Interactive configuration complete!")
        # print(f"\n📊 Comparison with legacy approach:")
        # print(f"   Legacy: 500+ lines of manual configuration")
        # print(f"   Interactive: Guided step-by-step process")
        # print(f"   Time saved: ~20-25 minutes")
        # print(f"   Error reduction: Validation at each step")

    except Exception as e:
        print(f"\n❌ Failed to save configurations: {e}")

else:
    print("\n⚠️  No configurations to save. Please generate configs first.")

2025-11-10 04:09:44,737 - WARNING - Config SystemPromptConfig has no categorize_fields method
2025-11-10 04:09:44,738 - WARNING - Config OutputFormatConfig has no categorize_fields method
2025-11-10 04:09:44,738 - WARNING - Config InstructionConfig has no categorize_fields method
2025-11-10 04:09:44,740 - WARNING - Config LabelConfig has no categorize_fields method
2025-11-10 04:09:44,741 - WARNING - Config RulesetDefinitionList has no categorize_fields method
2025-11-10 04:09:44,743 - INFO - Merging and saving 6 configs to /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-10 04:09:44,744 - INFO - Collecting field information for 6 configs (6 processing configs)
2025-11-10 04:09:44,745 - WARNING - Config SystemPromptConfig has no categorize_fields method
2025-11-10 04:09:44,745 - WARNING - Config OutputFormatConfig has no categorize_fields method
2025-11-10 04:09:44,746 - WARNING - Config InstructionCon

Saving configurations to: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json

✅ Configuration saved successfully!
   File: /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
   Size: 58.4 KB


### Test if we can load it

In [40]:
from cursus.steps.configs.config_dummy_data_loading_step import DummyDataLoadingConfig
from cursus.steps.configs.config_tabular_preprocessing_step import (
    TabularPreprocessingConfig,
)
from cursus.steps.configs.config_bedrock_prompt_template_generation_step import (
    BedrockPromptTemplateGenerationConfig,
)
from cursus.steps.configs.config_bedrock_batch_processing_step import (
    BedrockBatchProcessingConfig,
)
from cursus.steps.configs.config_label_ruleset_generation_step import (
    LabelRulesetGenerationConfig,
)
from cursus.steps.configs.config_label_ruleset_execution_step import (
    LabelRulesetExecutionConfig,
)

In [41]:
from cursus.steps.configs.utils import load_configs

In [42]:
CONFIG_CLASSES = {
    "DummyDataLoadingConfig": DummyDataLoadingConfig,
    "BedrockPromptTemplateGenerationConfig": BedrockPromptTemplateGenerationConfig,
    "BedrockBatchProcessingConfig": BedrockBatchProcessingConfig,
    "TabularPreprocessingConfig": TabularPreprocessingConfig,
    "LabelRulesetGenerationConfig": LabelRulesetGenerationConfig,
    "LabelRulesetExecutionConfig": LabelRulesetExecutionConfig,
}

In [43]:
loaded_configs = load_configs(str(config_path), CONFIG_CLASSES)

2025-11-10 04:09:59,829 - INFO - Loading configs from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-10 04:09:59,830 - INFO - Loading configuration from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-10 04:09:59,831 - WARNING - Could not find class InstructionConfig
2025-11-10 04:09:59,832 - WARNING - Could not find class OutputFormatConfig
2025-11-10 04:09:59,833 - WARNING - Could not find class SystemPromptConfig
2025-11-10 04:09:59,833 - WARNING - Could not find class LabelConfig
2025-11-10 04:09:59,833 - WARNING - Could not find class RulesetDefinitionList
2025-11-10 04:09:59,834 - INFO - Successfully loaded configuration from /home/ec2-user/SageMaker/AmazonSageMaker-lukexie-sagemaker-bsm-repo/pipelines/rnr_pytorch_bedrock/pipeline_config/config.json
2025-11-10 04:09:59,834 - INFO - Successfully loaded configs 

In [44]:
len(loaded_configs)

6

## Summary

This notebook demonstrates the **DAGConfigFactory** approach to pipeline configuration:

### ✅ **Benefits Achieved**

1. **Reduced Complexity**: From 500+ lines of manual config to guided workflow
2. **Base Config Inheritance**: Set common fields once, inherit everywhere
3. **Step-by-Step Guidance**: Clear requirements for each configuration step
4. **Validation**: Comprehensive validation prevents configuration errors
5. **Reusable DAG**: Pipeline structure defined once, reused across environments

### 🔄 **Workflow Comparison**

| Aspect | Legacy Approach | Interactive Approach |
|--------|----------------|---------------------|
| **Lines of Code** | 500+ manual lines | Guided step-by-step |
| **Time Required** | 30+ minutes | 10-15 minutes |
| **Error Rate** | High (manual entry) | Low (validation) |
| **Reusability** | Copy-paste heavy | DAG-driven |
| **Maintenance** | Manual updates | Automatic inheritance |

### 🚀 **Next Steps**

The generated configuration file can now be used with the existing pipeline compiler:

```python
# Use with pipeline compiler (from demo_pipeline.ipynb)
from cursus.core.compiler.dag_compiler import PipelineDAGCompiler

dag_compiler = PipelineDAGCompiler(
    config_path=config_path,
    sagemaker_session=pipeline_session,
    role=role
)

# Compile DAG to pipeline
template_pipeline, report = dag_compiler.compile_with_report(dag=dag)
```

The interactive configuration approach transforms the user experience from complex manual setup to an intuitive, guided workflow while maintaining full compatibility with the existing cursus infrastructure.